# ZTF Variable Marshal's API

In this notebook, we will programmatically interact with the ZTF Variable Marshal (ZVM) using its RESTful API. To simplify usage, the `zvm` `python` module

In [1]:
from IPython.core.display import display, HTML, JSON

from zvm import zvm
import json

For security, let us store the access credentials in a local `json` file `secrets_zvm.json`:

In [2]:
secrets = {
    "zvm": {
        "protocol": "http",
        "host": "localhost",
        "port": 8000,
        "username": "admin",
        "password": "admin"
    }
}

with open('secrets_zvm.json', 'w') as f:
    json.dump(secrets, f)

Load the credentials and initialize a `zvm` object:

In [3]:
with open('secrets_zvm.json', 'r') as f:
    secrets = json.load(f)

z = zvm(**secrets['zvm'], verbose=False)
print(f'Connection OK: {z.check_connection()}')

Connection OK: True


## Interact with ZVM

### Programs

Get programs:

In [4]:
r = z.api(endpoint='programs', method='get', data={'format': 'json'})
display(JSON(r, expanded=True))

<IPython.core.display.JSON object>

Add new program:

In [5]:
r = z.api(endpoint='programs', method='put', data={'program_name': 'variables', 
                                                   'program_description': 'dummy program'})
display(JSON(r, expanded=True))

<IPython.core.display.JSON object>

**todo:** `post` and `delete`

### Sources

Get source data in `JSON` format:

In [6]:
source_id = 'ZTFS1922e'
r = z.api(endpoint=f'sources/{source_id}', method='get', data={'format': 'json'})
display(JSON(r, expanded=False))

<IPython.core.display.JSON object>

Save a source to `zvm_program_id=1` on the Marshal given ($R.A.$, $Decl.$), automatically merging with ZTF sources within $2''$ (provided such exist):

In [7]:
ra, dec = 332.202248, -2.0567986
zvm_program_id = 1
r = z.api(endpoint='sources', method='put', data={'ra': ra, 'dec': dec, 
                                                  'zvm_program_id': zvm_program_id, 
                                                  'automerge': True})
display(JSON(r, expanded=False))

<IPython.core.display.JSON object>

Delete that source:

In [8]:
source_id = r['result']['_id']
# source_id = 'ZTFS1922g'

In [9]:
r = z.api(endpoint=f'sources/{source_id}', method='delete', data={'source_id': source_id})
display(JSON(r, expanded=False))

<IPython.core.display.JSON object>

---
Save a source to `zvm_program_id=1` on the Marshal given the ZTF `_id`, without automatically merging with ZTF sources within $2''$:

In [10]:
ztf_source_id = 10768392087896
zvm_program_id = 1
r = z.api(endpoint='sources', method='put', data={'_id': ztf_source_id, 
                                                  'zvm_program_id': zvm_program_id, 
                                                  'automerge': False})
display(JSON(r, expanded=False))

<IPython.core.display.JSON object>

In [11]:
source_id = r['result']['_id']

Merge the saved source with a ZTF source given its `_id`. Note that when `ztf_source_id` was previously saved for the source, it will be replaced.

In [12]:
ztf_source_id = 10768391017762
r = z.api(endpoint=f'sources/{source_id}', method='post', data={'source_id': source_id,
                                                                'action': 'merge',
                                                                '_id': ztf_source_id})
display(JSON(r, expanded=False))

<IPython.core.display.JSON object>

Add a light curve to the source:

In [13]:
lc = {"telescope": "KPNO:2.1m",
      "instrument": "KPED",
      "filter": "V",
      "id": 123456789,
      "lc_type": "temporal",
      "comment": "example fake data",
      "data": [
          {"mjd": 58400.0, "hjd": 2458400.5, "mag": 19.0, "magerr": 0.05},
          {"mjd": 58401.0, "hjd": 2458401.5, "mag": 19.1, "magerr": 0.05},
          {"mjd": 58402.5, "hjd": 2458403.0, "mag": 18.9, "magerr": 0.05},
          {"mjd": 58404.0, "hjd": 2458404.5, "mag": 19.2, "magerr": 0.05},
          {"mjd": 58407.0, "hjd": 2458407.5, "mag": 18.7, "magerr": 0.05}
      ]
}

In [14]:
r = z.api(endpoint=f'sources/{source_id}', method='post', data={'source_id': source_id,
                                                                'action': 'upload_lc',
                                                                'data': lc})
display(JSON(r, expanded=False))

<IPython.core.display.JSON object>

Delete that light curve:

In [15]:
r = z.api(endpoint=f'sources/{source_id}', method='get', data={'format': 'json'})
lc_id = r['lc'][-1]['_id']
r = z.api(endpoint=f'sources/{source_id}', method='post', data={'source_id': source_id,
                                                                'action': 'remove_lc',
                                                                'lc_id': lc_id})
display(JSON(r, expanded=False))

<IPython.core.display.JSON object>

Add a spectrum:

In [16]:
spec = {
  "telescope": "PO:5m",
  "instrument": "DBSP",
  "filter": "V",
  "mjd": 58400.0,
  "wavelength_unit": "A",
  "flux_unit": "arbitrary",
  "comment": "random DBSP spectrum",
  "data": [
    {"wavelength": 3400.0, "flux": 20688.13, "fluxerr": 0.5},
    {"wavelength": 3401.07, "flux": 23494.96, "fluxerr": 0.5},
    {"wavelength": 3402.14, "flux": 23069.39, "fluxerr": 0.5},
    {"wavelength": 3403.21, "flux": 22004.25, "fluxerr": 0.5},
    {"wavelength": 3404.28, "flux": 23031.14, "fluxerr": 0.5},
  ]
}

In [17]:
r = z.api(endpoint=f'sources/{source_id}', method='post', data={'source_id': source_id,
                                                                'action': 'upload_spectrum',
                                                                'data': spec})
display(JSON(r, expanded=False))

<IPython.core.display.JSON object>

Delete that spectrum:

In [59]:
r = z.api(endpoint=f'sources/{source_id}', method='get', data={'format': 'json'})
spectrum_id = r['spec'][-1]['_id']
r = z.api(endpoint=f'sources/{source_id}', method='post', data={'source_id': source_id,
                                                                'action': 'remove_spectrum',
                                                                'spectrum_id': spectrum_id})
display(JSON(r, expanded=False))

<IPython.core.display.JSON object>

---
Transfer the source to a different `zvm_program_id`, then add a note on that, source type, period, and source flags, and re-run cross-matching with external catalogs:

In [18]:
r = z.api(endpoint=f'sources/{source_id}', method='post', data={'source_id': source_id,
                                                                'action': 'add_note',
                                                                'note': 'sup dude'})
r = z.api(endpoint=f'sources/{source_id}', method='post', data={'source_id': source_id,
                                                                'action': 'add_source_type',
                                                                'source_type': 'Eclipsing binary'})
r = z.api(endpoint=f'sources/{source_id}', method='post', data={'source_id': source_id,
                                                                'action': 'add_period',
                                                                'period': '1',
                                                                'period_unit': 'Days'})
r = z.api(endpoint=f'sources/{source_id}', method='post', data={'source_id': source_id,
                                                                'action': 'add_source_flags',
                                                                'source_flags': ['Eclipsing']})
r = z.api(endpoint=f'sources/{source_id}', method='post', data={'source_id': source_id,
                                                                'action': 'run_cross_match'})

### Query sources saved on the ZVM

Note: The interface is *almost* exactly the same as in the `penquins` library. The source data are stored in the `sources` collection in ZVM's `mongodb`.

In [5]:
q = {"query_type": "find",
     "query": {
         "catalog": "sources",
         "filter": {},
         "projection": {'_id': 1}
     },
     "kwargs": {
         "sort": [('_id', -1)],
         "limit": 5 
     }
    }
r = z.query(query=q)
data = r['result']['result_data']['query_result']
# display(JSON(r, expanded=True))
display(JSON(data, expanded=True))

<IPython.core.display.JSON object>

In [6]:
q = {"query_type": "cone_search",
     "object_coordinates": {
         "radec": '[(332.202248, -2.0567986)]', 
         "cone_search_radius": "2",
         "cone_search_unit": "arcsec"
     },
     "catalogs": {
         "sources": {
             "filter": {
                 "zvm_program_id": {"$in": [1, 2]}
             },
             "projection": {
                 "_id": 1
             }
         }
     },
     "kwargs": {
         "limit": 3
     }
     }
r = z.query(query=q)
# r
data = r['result']['result_data']
display(JSON(data, expanded=True))

<IPython.core.display.JSON object>